<a href="https://colab.research.google.com/github/BioCellColman/BioCell/blob/master/Notebooks/Model/U_Net_base_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
use_local = False

In [0]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import skimage
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
from tensorflow.keras import utils
import tensorflow as tf

In [3]:
if not use_local:
    base_dir = 'drive/My Drive/Colab Notebooks/ML course/final_project/dataset/'
    from google.colab import drive

    drive.mount('/content/drive')
else:
    base_dir = 'C:\\Users\lange\\Dev\\ML\\final_project\\dataset\\'
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
    config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using local runtime!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#U-Net Model

In [0]:
def unet(pretrained_weights = None,input_size = (512,512,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(4, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


#Data generator

In [0]:
class DataGenerator(utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path, ext, batch_size=32, dim=(572,572), n_channels=4,
                 shuffle=True, seed=1, is_image=False):
        'Initialization'
        self.dim = dim
        self.path = path
        self.batch_size = batch_size
        self.list_IDs = [file for file in os.listdir(path) if file.endswith(ext)]
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.is_image = is_image
        self.ext = ext

        print('Found ' + str(len(self.list_IDs)) + ' files.')

        np.random.seed(seed)
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X = self.__data_generation(list_IDs_temp)

        return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        if (self.is_image and self.n_channels == 1):
            X = np.empty((self.batch_size, *self.dim))
        else:
            X = np.empty((self.batch_size, self.n_channels, *self.dim))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            if self.is_image:
                X[i,] = io.imread(self.path + '/' + ID)
            else:
                X[i,] = np.load(self.path + '/' + ID, allow_pickle=True)

        return X

#Data

In [0]:
from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans

Sky = [128,128,128]
Building = [128,0,0]
Pole = [192,192,128]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])


def adjustData(img,mask,flag_multi_class=False,num_class=0):
    # mask doesn't require normalization for now, it's values are 1/0
    img = img / (2**16 - 1.0)

    return (img,mask)


def trainGenerator(batch_size, train_path, image_folder, mask_folder, image_color_mode = "grayscale",
                    mask_color_mode = "grayscale", image_save_prefix = "image", mask_save_prefix = "mask",
                    flag_multi_class = False, num_class = 2, save_to_dir = None, target_size = (572,572),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_generator = DataGenerator(
        train_path + '/' + image_folder,
        '.png',
        batch_size,
        n_channels=1,
        seed=seed,
        is_image=True)
    mask_generator = DataGenerator(
        train_path + '/' + mask_folder,
        '.npy',
        batch_size,
        seed=seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)

        mask = np.swapaxes(mask, 1, 2)
        mask = np.swapaxes(mask, 2, 3)

        ## TODO: Temporary size fix, remove later
        img = tempFixSize(img)
        mask = tempFixSize(mask, label=True)

        yield (img,mask)



def testGenerator(test_path,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    # TODO fix this generator

    # for i in range(num_image):
    #     img = io.imread(os.path.join(test_path,"%d.png"%i),as_gray = as_gray)
    #     img = img / 255
    #     img = trans.resize(img,target_size)
    #     img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
    #     img = np.reshape(img,(1,)+img.shape)
    #     yield img
    pass


def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255



def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        img = skimage.img_as_ubyte(img)
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

def tempFixSize(img, label=False):
    # if label
    if label:
        img = img[:,30:542,30:542,:]
    # if image
    else:
        img = img[:,30:542,30:542]

    return img

#Main

In [7]:
!ls "drive/My Drive/Colab Notebooks/ML course/final_project/dataset/"
!rm "drive/My Drive/Colab Notebooks/ML course/final_project/dataset/unet_biocell.hdf5"

biocell  membrane
rm: cannot remove 'drive/My Drive/Colab Notebooks/ML course/final_project/dataset/unet_biocell.hdf5': No such file or directory


In [8]:
myGene = trainGenerator(1, base_dir + 'biocell/train', 'images', 'labels', save_to_dir = None)

model = unet()
model_checkpoint = ModelCheckpoint(
    base_dir + 'unet_biocell.hdf5',
     monitor='loss',verbose=1,
     save_best_only=True)
model.fit(myGene,steps_per_epoch=300,epochs=1,callbacks=[model_checkpoint])

# testGene = testGenerator(base_dir + 'membrane/test')
# results = model.predict(testGene,30,verbose=1)
# saveResult(base_dir + 'membrane/test',results)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________